In [1]:
import os
import openai

In [2]:
from dotenv import load_dotenv
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
from openai import AsyncOpenAI

In [4]:
from llama_index import SimpleDirectoryReader

docs = SimpleDirectoryReader(
    input_files = ["./ce-little-red-story-pcp-115014.pdf"]
).load_data() # one doc per page 

# Basic RAG Pipeline

In [5]:
from llama_index import Document

doc = Document(text = '\n\n'.join([doc.text for doc in docs])) #merge the docs into one doc

In [6]:
from llama_index import VectorStoreIndex
from llama_index import ServiceContext
from llama_index.llms import OpenAI

llm = OpenAI(model = 'gpt-3.5-turbo', temperature = 0.1)
service_context = ServiceContext.from_defaults(
            llm = llm,
            embed_model = 'local:BAAI/bge-small-en-v1.5' # hugging face model
)
index = VectorStoreIndex.from_documents([doc], service_context = service_context)

In [7]:
query_engine = index.as_query_engine() # send user query and compare it against data

In [8]:
response = query_engine.query(
    'What is the story about?')

In [9]:
print(str(response))

The story is about a little girl named Little Red Riding Hood who is sent by her mother to take a basket of sweet cherry pies to her grandmother's house. Along the way, she encounters a thorn that holds her cloak, and a tall dark shape that turns out to be the Big Bad Wolf. Unaware of the wolf's true identity, Little Red Riding Hood continues on her journey and arrives at her grandmother's cottage, only to find the wolf disguised as her grandmother. The wolf reveals its true identity and tries to eat Little Red Riding Hood, but she is saved by the Wise Owl and the Storyland Vets. In the end, the wolf is put to sleep and Grandma is found safe, but her two pointy shoes are accidentally left inside the wolf's stomach. The story teaches the lesson of not speaking to strangers, particularly the Big Bad Wolf.


# Evaluating setup using TruLens

## a set of questions to evaluate the model against

In [10]:
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        item = line.strip() #remove new line character and convert it to integer
        print(item)
        eval_questions.append(item)

What is the name of the main character?
What was in Red Riding Hood basket?
Whom did Red Riding Hood went to visit?
Where did the grandma live?
Who ate the grandma?
Who helped Red Riding Hood to rescue the grandma?
What happened to the wolf at the end?


In [11]:
new_question = 'what was the morale of the story?'
eval_questions.append(new_question)
eval_questions

['What is the name of the main character?',
 'What was in Red Riding Hood basket?',
 'Whom did Red Riding Hood went to visit?',
 'Where did the grandma live?',
 'Who ate the grandma?',
 'Who helped Red Riding Hood to rescue the grandma?',
 'What happened to the wolf at the end?',
 'what was the morale of the story?']

## Setting up the model

In [16]:
from trulens_eval import Tru
tru = Tru()

tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [42]:
from utils_trulens import get_prebuilt_trulens_recorder
tru_recorder = get_prebuilt_trulens_recorder(query_engine, app_id = 'Direct Query Engine')

In [43]:
with tru_recorder as recording:
    for question in eval_questions:
        response = query_engine.query(question)

In [44]:
records, feedback = tru.get_records_and_feedback(app_ids = [])

In [45]:
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Groundedness,Context Relevance,Answer Relevance_calls,Groundedness_calls,Context Relevance_calls,latency,total_tokens,total_cost
0,Sentence window query engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_d73ad65a2e8c2a832317bb0b41b4ab80,"""What is the name of the main character?""","""The name of the main character is Little Red ...",-,"{""record_id"": ""record_hash_d73ad65a2e8c2a83231...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-19T19:35:30.662360"", ""...",2023-12-19T19:35:35.684319,1.0,1.0,0.1,[{'args': {'prompt': 'What is the name of the ...,[{'args': {'source': '“Wait! You’re not my gr...,[{'args': {'prompt': 'What is the name of the ...,5,373,0.000566
1,Sentence window query engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_bac3b702d567dc894b00290981574ae0,"""What was in Red Riding Hood basket?""","""Sweet cherry pies.""",-,"{""record_id"": ""record_hash_bac3b702d567dc894b0...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-19T19:35:36.163743"", ""...",2023-12-19T19:35:41.385144,0.2,1.0,0.1,[{'args': {'prompt': 'What was in Red Riding H...,[{'args': {'source': 'www.scholastic.co.ukhttp...,[{'args': {'prompt': 'What was in Red Riding H...,5,327,0.000492
2,Sentence window query engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_21da6935659d39bdd4ce07ece1677eaa,"""Whom did Red Riding Hood went to visit?""","""Red Riding Hood went to visit her Grandma.""",-,"{""record_id"": ""record_hash_21da6935659d39bdd4c...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-19T19:35:41.845506"", ""...",2023-12-19T19:35:47.161259,1.0,1.0,0.5,[{'args': {'prompt': 'Whom did Red Riding Hood...,[{'args': {'source': 'www.scholastic.co.ukhttp...,[{'args': {'prompt': 'Whom did Red Riding Hood...,5,378,0.000571
3,Sentence window query engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_7e6bf466f837be66b7cb1f7f42c2b1d9,"""Where did the grandma live?""","""The grandma lived in a cottage.""",-,"{""record_id"": ""record_hash_7e6bf466f837be66b7c...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-19T19:35:47.605333"", ""...",2023-12-19T19:35:52.934423,1.0,1.0,0.0,[{'args': {'prompt': 'Where did the grandma li...,[{'args': {'source': '“I am in bed with a nast...,[{'args': {'prompt': 'Where did the grandma li...,5,395,0.000596
4,Sentence window query engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_7415a142a119cca56e14b75d837febe0,"""Who ate the grandma?""","""The Big Bad Wolf ate the grandma.""",-,"{""record_id"": ""record_hash_7415a142a119cca56e1...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-19T19:35:53.425317"", ""...",2023-12-19T19:35:58.821892,1.0,1.0,0.0,"[{'args': {'prompt': 'Who ate the grandma?', '...",[{'args': {'source': '“What big eyes you’ve go...,"[{'args': {'prompt': 'Who ate the grandma?', '...",5,477,0.000719


In [47]:
tru.get_leaderboard(app_ids = [])

,Groundedness,Answer Relevance,Context Relevance,latency,total_cost
app_id,,,,,
Direct Query Engine,1.00,1.0,0.00000,4.777778,0.001704
Sentence window query engine,0.75,0.9,0.14375,5.000000,0.000605


In [46]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   URL: http://localhost:8501



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

# Advanced RAG pipeline

## Sentence window retrieval

In [23]:
from llama_index.llms import OpenAI

llm = OpenAI(model = 'gpt-3.5-turbo', temperature = 0.1)

In [28]:
from utils_trulens import build_sentence_window_index
# retreive multiple sentences instead of one
sentence_index = build_sentence_window_index(
    doc,
    llm,
    embed_model = 'local:BAAI/bge-small-en-v1.5',
    save_dir = 'sentence_index'
)

In [30]:
from utils_trulens import get_sentence_window_query_engine
sentence_window_engine = get_sentence_window_query_engine(sentence_index)

config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

C:\Users\ergo_\Anaconda3\envs\llm\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ergo_\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

In [32]:
response = sentence_window_engine.query(
    'what was the morale of the story?'
)
print(str(response))

The morale of the story is to be cautious and not trust strangers, as they may have ulterior motives.


In [33]:
response = query_engine.query(
    'what was the morale of the story?'
)
print(str(response))

The morale of the story is to never stop and speak to the Big Bad Wolf.


In [35]:
tru.reset_database()

tru_recorder_sentence_window = get_prebuilt_trulens_recorder(
    sentence_window_engine,
    app_id = 'Sentence window query engine'
)

In [36]:
for question in eval_questions:
    with tru_recorder_sentence_window as recording:
        response = sentence_window_engine.query(question)
        print(question)
        print(str(response))

What is the name of the main character?
The name of the main character is Little Red Riding Hood.
What was in Red Riding Hood basket?
Sweet cherry pies.
Whom did Red Riding Hood went to visit?
Red Riding Hood went to visit her Grandma.
Where did the grandma live?
The grandma lived in a cottage.
Who ate the grandma?
The Big Bad Wolf ate the grandma.
Who helped Red Riding Hood to rescue the grandma?
There is no information provided in the given context about who helped Red Riding Hood rescue her grandma.
What happened to the wolf at the end?
The wolf was sewn up after the Storyland Vets found Grandma safe and well inside his tummy. However, they accidentally left Grandma's two pointy shoes inside him.
what was the morale of the story?
The morale of the story is to be cautious and not talk to strangers.


In [38]:
tru.get_leaderboard(app_ids = [])

,Groundedness,Answer Relevance,Context Relevance,latency,total_cost
app_id,,,,,
Sentence window query engine,0.75,0.9,0.14375,5.0,0.000605


## Auto_merging retrieval

In [54]:
from utils_trulens import build_automerging_index

automerging_index = build_automerging_index(
        doc,
        llm,
        embed_model="local:BAAI/bge-small-en-v1.5",
        save_dir="merging_index"
        )

AttributeError: 'tuple' object has no attribute 'id_'

In [ ]:
from utils_trulens import get_automerging_query_engine
automerging_query_engine = get_automerging_query_engine(
    automerging_index
)

In [ ]:
response = automerging_query_engine .query(
    'what was the morale of the story?'
)
print(str(response))

In [52]:
tru.reset_database()

tru_recorder_automerging = get_prebuilt_trulens_recorder(automerging_query_engine,
                                                         app_id="Automerging Query Engine")

In [53]:
for question in eval_questions:
    with tru_recorder_automerging as recording:
        response = automerging_query_engine.query(question)
        print(question)
        print(response)

A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x207d7429750 is calling an instrumented method <function BaseQueryEngine.query at 0x00000207BBFB3130>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x207b7683370) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x207d7429750 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x00000207BF87A7A0>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x207b7683370) using this function.
A new object of type <class 'llama_index.retrievers.auto_merging_retriever.AutoMergingRetriever'> at 0x207d7514910 is calling an instrumented method <function BaseRetriever.retrieve at 0x00000207BBFB23B0>. The path of this call may be incorrect.
Guessing path of new object is app.retriever based on other obje

> Merging 5 nodes into parent node.
> Parent node id: 374b0bb9-3339-43c0-904e-5eff4ace2d9a.
> Parent node text: www.scholastic.co.ukhttp://www.scholastic.co.uk/
Fairytale Kingdom
The Story of Little Red Riding...

> Merging 4 nodes into parent node.
> Parent node id: 5e1e21cd-474e-425b-8054-46fc312af0ca.
> Parent node text: www.scholastic.co.ukhttp://www.scholastic.co.uk/
 “Grandma”, gasped Little Red Riding Hood. “What...

> Merging 2 nodes into parent node.
> Parent node id: d8868ccb-da68-470e-b752-b6544fdbe3fe.
> Parent node text: “Let me in, Grandma,” she called merrily. “I have brought you a basket of sweet cherry pies.”
 “L...

> Merging 2 nodes into parent node.
> Parent node id: 7cf2567c-9525-419a-8072-d6efb6cf9895.
> Parent node text: www.scholastic.co.ukhttp://www.scholastic.co.uk/
Fairytale Kingdom
The Story of Little Red Riding...

> Merging 1 nodes into parent node.
> Parent node id: daeae9c7-0679-4091-aabd-c4296e87d252.
> Parent node text: www.scholastic.co.ukhttp://www.s

A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x207d7428520 is calling an instrumented method <function BaseSynthesizer.synthesize at 0x00000207BCCA5120>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x207b7683790) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x207d7428520 is calling an instrumented method <function CompactAndRefine.get_response at 0x00000207BCCA4550>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x207b7683790) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x207d7428520 is calling an instrumented method <function Refine.get_response at 0x00000207BD5F8EE0>. The path of this call may be incorrect.
Guessing path

What is the name of the main character?
The main character's name is Little Red Riding Hood.


A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x207d7429750 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x00000207BF87A7A0>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x207b7683370) using this function.


...

What happened to the wolf at the end?
The wolf was put to sleep by the Storyland Vets and Grandma was found safe and well inside its tummy. However, when the wolf was sewn up again, Grandma's two pointy shoes were accidentally left inside. Now, whenever the wolf feels hungry, the shoes dance and prance, causing him to howl, and he never dreams of eating a grandma again.


A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x207d7429750 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x00000207BF87A7A0>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x207b7683370) using this function.


> Merging 4 nodes into parent node.
> Parent node id: 5e1e21cd-474e-425b-8054-46fc312af0ca.
> Parent node text: www.scholastic.co.ukhttp://www.scholastic.co.uk/
 “Grandma”, gasped Little Red Riding Hood. “What...

> Merging 2 nodes into parent node.
> Parent node id: d8868ccb-da68-470e-b752-b6544fdbe3fe.
> Parent node text: “Let me in, Grandma,” she called merrily. “I have brought you a basket of sweet cherry pies.”
 “L...

> Merging 5 nodes into parent node.
> Parent node id: 374b0bb9-3339-43c0-904e-5eff4ace2d9a.
> Parent node text: www.scholastic.co.ukhttp://www.scholastic.co.uk/
Fairytale Kingdom
The Story of Little Red Riding...

> Merging 1 nodes into parent node.
> Parent node id: daeae9c7-0679-4091-aabd-c4296e87d252.
> Parent node text: www.scholastic.co.ukhttp://www.scholastic.co.uk/
 “Grandma”, gasped Little Red Riding Hood. “What...

> Merging 2 nodes into parent node.
> Parent node id: 7cf2567c-9525-419a-8072-d6efb6cf9895.
> Parent node text: www.scholastic.co.ukhttp://www.s

A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x207d7428520 is calling an instrumented method <function CompactAndRefine.get_response at 0x00000207BCCA4550>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x207b7683790) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x207d7428520 is calling an instrumented method <function Refine.get_response at 0x00000207BD5F8EE0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x207b7683790) using this function.


what was the morale of the story?
The morale of the story is to never stop and speak to the Big Bad Wolf.


In [ ]:
tru.get_leaderboard(app_ids=[])